# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import ResNet38 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/CNN_finetune_ex2/config.yaml'

In [7]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [8]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # model out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['param']['sample_rate'],
                window_size=config['param']['window_size'],
                hop_size=config['param']['hop_size'],
                mel_bins=config['param']['mel_bins'],
                fmin=config['param']['fmin'],
                fmax=config['param']['fmax'],
                classes_num=6
                )
    optimizer = optim.Adam(net.parameters())
    criterion = nn.CrossEntropyLoss()
    num_epochs = config['param']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # load pre-trained model
    pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_ROOT'])
    net.load_state_dict(pretrained_dict['model'], strict=False)
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, criterion, scheduler, num_epochs, writer, model_out_path)
    
    com.toc()

In [10]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [11]:
for machine_type in machine_types:
    run(machine_type, dev_paths)

2021-04-12 01:23:45,281 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2021-04-12 01:23:45,282 - 00_train.py - INFO - MAKE DATA_LOADER
2021-04-12 01:23:45,284 - 00_train.py - INFO - TRAINING
2021-04-12 01:23:48,734 - pytorch_modeler.py - INFO - train
  0%|          | 0/95 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 95/95 [01:08<00:00,  1.39it/s]
2021-04-12 01:24:56,886 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 10/10 [00:02<00:00,  4.06it/s]
2021-04-12 01:25:01,347 - pytorch_modeler.py - INFO - Save best model
2021-04-12 01:25:01,347 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 10/10 [00:02<00:00,  3.97it/s]
2021-04-12 01:25:03,867 - pytorch_modeler.py - INFO - epoch:1/100, train_losses:17.685401678085327, val_source_losses:1.840585, val_target_losses:1.768540, best_flag:True
2021-04-12 01:25:03,868 - pytorch_modeler.py - INFO - train
100%|██████████| 95/95 [01:08<00:00,  1.40it/s]
2021-04-12 01:26:11,963 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 10/10 [00:02<00:00,  4.00it/s]
2021-04-12 01:26:16,445 - pytorch_modeler.py - INFO - Save best model
2021-04-12 01:26:16,446 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 10/10 [00:02<00:00,  4.02it/s]
2021-04-12 01:26:18,934 - pytorch_modeler.py - INFO - epoch:2/100, train_

100%|██████████| 10/10 [00:02<00:00,  4.21it/s]
2021-04-12 01:42:55,818 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 10/10 [00:02<00:00,  4.21it/s]
2021-04-12 01:42:58,194 - pytorch_modeler.py - INFO - epoch:16/100, train_losses:8.742170967161655, val_source_losses:1.074699, val_target_losses:0.874217, best_flag:False
2021-04-12 01:42:58,197 - pytorch_modeler.py - INFO - train
100%|██████████| 95/95 [01:05<00:00,  1.45it/s]
2021-04-12 01:44:03,515 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 10/10 [00:02<00:00,  4.22it/s]
2021-04-12 01:44:07,785 - pytorch_modeler.py - INFO - Save best model
2021-04-12 01:44:07,786 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 10/10 [00:02<00:00,  4.22it/s]
2021-04-12 01:44:10,156 - pytorch_modeler.py - INFO - epoch:17/100, train_losses:12.052836209535599, val_source_losses:0.876917, val_target_losses:1.205284, best_flag:True
2021-04-12 01:44:10,157 - pytorch_modeler.py - INFO - train
100%|██████████| 95/95 [0

2021-04-12 02:00:38,946 - pytorch_modeler.py - INFO - train
100%|██████████| 95/95 [01:05<00:00,  1.46it/s]
2021-04-12 02:01:44,231 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 10/10 [00:02<00:00,  4.20it/s]
2021-04-12 02:01:46,614 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 10/10 [00:02<00:00,  4.22it/s]
2021-04-12 02:01:48,988 - pytorch_modeler.py - INFO - epoch:32/100, train_losses:8.38233669102192, val_source_losses:0.800164, val_target_losses:0.838234, best_flag:False
2021-04-12 02:01:48,989 - pytorch_modeler.py - INFO - train
100%|██████████| 95/95 [01:05<00:00,  1.45it/s]
2021-04-12 02:02:54,326 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 10/10 [00:02<00:00,  4.24it/s]
2021-04-12 02:02:56,688 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 10/10 [00:02<00:00,  4.16it/s]
2021-04-12 02:02:59,096 - pytorch_modeler.py - INFO - epoch:33/100, train_losses:7.605737369507551, val_source_losses:0.887632, val_target_losses:0.76057

...

elapsed time: 7145.669826031 [sec]


2021-04-12 07:22:55,306 - pytorch_modeler.py - INFO - train
  0%|          | 0/95 [00:00<?, ?it/s]

use: cuda:0


  1%|          | 1/95 [00:11<18:22, 11.73s/it]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.77 GiB total capacity; 8.77 GiB already allocated; 65.31 MiB free; 8.91 GiB reserved in total by PyTorch)

In [ ]:
machine_type = 'pump'
input_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
ext_data = pd.read_pickle(input_path)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['train']['features'], cmap='jet')
plt.colorbar()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['valid_source']['features'], cmap='jet')
plt.colorbar()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['train']['features'], cmap='jet')
plt.colorbar()

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)

In [ ]:
print(M_means.shape)
plt.imshow(M_means, aspect='auto', cmap='jet')
plt.title(phase)
plt.colorbar()
plt.show()